In [12]:
import pandas as pd
import numpy as np

from itertools import chain

from pathlib import Path
from functools import reduce

In [13]:
path = Path('./corona_data_sets_modified') # use your path
all_files = list(path.rglob("*.csv"))

In [14]:
d = {}

for p in all_files:
    df = pd.read_csv(p, index_col=None, header=0, converters={'ags2': str, 'ags5': str})
    #df = df.drop(['bundesland', 'kreis', '_id'], 1)
    try:
        df = df.drop(['id'], 1) # only few dbs have the 'id' column
    except:
        pass
    d[p.stem] = df

In [15]:
d.keys()

dict_keys(['reachability_data_modified', 'training_situation', 'hospital_data', 'households', 'traffic', 'publicFinances', 'population', 'commuter_data_modified', 'tourism_data_modified', 'company_information', 'point_of_interest', 'spatialPlanning', 'land_use_data', 'private_finance'])

In [16]:
{k: len(set(v["ags5"])) for k, v in d.items()}

{'reachability_data_modified': 401,
 'training_situation': 401,
 'hospital_data': 401,
 'households': 401,
 'traffic': 401,
 'publicFinances': 401,
 'population': 401,
 'commuter_data_modified': 401,
 'tourism_data_modified': 401,
 'company_information': 401,
 'point_of_interest': 401,
 'spatialPlanning': 401,
 'land_use_data': 401,
 'private_finance': 401}

In [17]:
dataFrames = [v for v in d.values()]
col = []

total_number_of_columns = 1 # for ags2 (ags 5 becomes index, so not counted as a column)
for df in dataFrames:
    total_number_of_columns += len(df.columns) - 2 # dono't double count ags2 and ags 5 for every table
    col.append(df.columns.values.tolist())
    
print('Total number of columns: ', total_number_of_columns)


Total number of columns:  180


In [18]:
df_final = dataFrames[0].sort_values(by=['ags5'])
for df in dataFrames[1:]:
    for col in df.columns:
        df_final[col] = df.sort_values(by=['ags5'])[col]

# set the index
df_final.set_index('ags5', inplace=True, drop=True)
df_final

,ags2,kreis,supermarkets_population,supermarkets_average_distance,public_transport availability,average_distance_bus_stop,average_distance_train_station,average_distance_public_transport,Unnamed: 0,number_of_students,...,median_income,purchasing_power_per_household,purchasing_power_per_person,debtor_quota,household_in_income_calss_1,household_in_income_calss_2,household_in_income_calss_3,household_in_income_calss_4,household_in_income_calss_5,household_in_income_calss_6
ags5,,,,,,,,,,,,,,,,,,,,,
01001,01,"Flensburg, Stadt",92,500,35,240,2901,240,0,11610,...,2986,34496,19556,16.0,13209,12385,13092,5441,3096,3497
01002,01,"Kiel, Landeshauptstadt",92,460,37,268,2037,265,1,23898,...,3304,35246,19612,12.1,22126,38523,39483,18567,11162,7841
01003,01,"Lübeck, Hansestadt",90,532,37,297,1927,294,2,20478,...,3036,37219,20820,15.1,22881,30159,32571,15805,10999,9039
01004,01,"Neumünster, Stadt",85,588,37,316,1648,313,3,10380,...,2842,38141,19561,17.9,7208,13291,10435,3812,3201,2805
10041,10,Regionalverband Saarbrücken,76,734,52,260,1804,255,318,30091,...,3431,37046,19917,14.4,31031,44117,53073,24122,13716,11119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
09776,09,Lindau (Bodensee),63,1056,36,461,3039,441,313,7001,...,3334,53011,27281,7.6,1899,4382,10928,7859,6828,10105
09777,09,Ostallgäu,59,1418,33,907,3726,837,314,13093,...,3231,52358,25107,6.3,4076,8392,16455,11687,10814,15803
09778,09,Unterallgäu,58,1369,32,855,4129,805,315,12022,...,3297,60441,27461,6.1,4051,6995,15103,12738,10743,15768


In [19]:
col

'household_in_income_calss_6'

In [20]:
df_final.columns

Index(['ags2', 'kreis', 'supermarkets_population',
       'supermarkets_average_distance', 'public_transport availability',
       'average_distance_bus_stop', 'average_distance_train_station',
       'average_distance_public_transport', 'Unnamed: 0', 'number_of_students',
       ...
       'median_income', 'purchasing_power_per_household',
       'purchasing_power_per_person', 'debtor_quota',
       'household_in_income_calss_1', 'household_in_income_calss_2',
       'household_in_income_calss_3', 'household_in_income_calss_4',
       'household_in_income_calss_5', 'household_in_income_calss_6'],
      dtype='object', length=162)

In [21]:
df_final.columns = df_final.columns.str.replace(' ', '_')

In [22]:
df_final = df_final.drop('kreis', 1)
df_arb = pd.read_csv('./corona_data_sets/arbeitsmarktentwicklung.csv', converters={'ags2': str, 'ags5': str}) 
df_arb
df_final = pd.merge(df_arb[["kreis", "ags5"]], df_final, on = "ags5")
df_final

,kreis,ags5,ags2,supermarkets_population,supermarkets_average_distance,public_transport_availability,average_distance_bus_stop,average_distance_train_station,average_distance_public_transport,Unnamed:_0,...,median_income,purchasing_power_per_household,purchasing_power_per_person,debtor_quota,household_in_income_calss_1,household_in_income_calss_2,household_in_income_calss_3,household_in_income_calss_4,household_in_income_calss_5,household_in_income_calss_6
0,"Flensburg, Stadt",01001,01,92,500,35,240,2901,240,0,...,2986,34496,19556,16.0,13209,12385,13092,5441,3096,3497
1,"Kiel, Landeshauptstadt",01002,01,92,460,37,268,2037,265,1,...,3304,35246,19612,12.1,22126,38523,39483,18567,11162,7841
2,"Lübeck, Hansestadt",01003,01,90,532,37,297,1927,294,2,...,3036,37219,20820,15.1,22881,30159,32571,15805,10999,9039
3,"Neumünster, Stadt",01004,01,85,588,37,316,1648,313,3,...,2842,38141,19561,17.9,7208,13291,10435,3812,3201,2805
4,Dithmarschen,01051,01,51,1864,35,448,3517,443,4,...,2914,46945,23486,12.8,11806,11575,17816,9873,7834,7676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,Saalfeld-Rudolstadt,16073,16,62,1423,37,341,2492,322,396,...,2466,39242,20959,8.3,12237,13267,19810,5653,3271,2547
397,Saale-Holzland-Kreis,16074,16,54,1677,38,343,3419,340,397,...,2288,42956,20631,7.6,5131,9216,14445,5254,3446,2379
398,Saale-Orla-Kreis,16075,16,54,1686,35,449,3172,444,398,...,2253,40511,20610,7.6,7388,7994,17082,4532,2501,1632
399,Greiz,16076,16,55,1654,38,608,2779,563,399,...,2233,40784,21275,7.8,11098,10354,18760,5324,3370,2287


In [23]:
len(df_final['kreis'].unique())

401

In [24]:
#df_final.to_csv('./corona_data_sets_modified/df_final.csv', index = False)